In [ ]:
!pip install pyspark requests

from pyspark.sql import SparkSession
from google.colab import drive
drive.mount('/content/drive')
import requests
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=e524221476e937f5a693855e57e8ec0a62e3c441037946293670d7953ae31400
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.appName("Bronze").getOrCreate()

In [ ]:
api_url = "https://api.openbrewerydb.org/breweries"

# Recebe os dados da API
response = requests.get(api_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")

In [ ]:
# Cria um DataFrame a partir dos dados JSON
df = spark.read.json(spark.sparkContext.parallelize([json.dumps(data)]))

# Exibe o DataFrame
df.show(50)

+--------------------+---------+---------+------------+----------------+-------------+--------------------+-----------------+------------------+--------------------+------------+-----------+--------------+--------------+--------------------+--------------------+
|           address_1|address_2|address_3|brewery_type|            city|      country|                  id|         latitude|         longitude|                name|       phone|postal_code|         state|state_province|              street|         website_url|
+--------------------+---------+---------+------------+----------------+-------------+--------------------+-----------------+------------------+--------------------+------------+-----------+--------------+--------------+--------------------+--------------------+
|      1716 Topeka St|     NULL|     NULL|       micro|          Norman|United States|5128df48-79fc-4f0...|      35.25738891|      -97.46818222|    (405) Brewing Co|  4058160490| 73069-8224|      Oklahoma|      

In [ ]:
# Realiza escrita no diretório do Drive
bronze_file_path = '/content/drive/My Drive/bronze.json'
df.write.mode("overwrite").json(bronze_file_path)